In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import urllib2
import csv

# open a browser
browser = Browser('chrome')
# browser.driver.maximize_window() #browser size maximize windows
browser.driver.set_window_size(2500, 3000)

# Visiting the page
browser.visit('http://nik.depkop.go.id/')
element = browser.find_by_xpath('//select[@id="MainContent_DropDownList1"]//option[@value="31"]').first #untuk pilih Provinsi
element.click()
element = browser.find_by_xpath('//select[@id="MainContent_DropDownList2"]//option[@value="75"]').first #untuk pilih Kota/Kab
element.click()

df = pd.DataFrame(columns = ['DATA KOPERASI', 'Koperasi', 'Nomor Badan Hukum Pendirian', 
                 'Tanggal Badan Hukum Pendirian', 'Nomor Perubahan Anggaran Dasar (Terbaru)', 
                 'Tanggal Perubahan Anggaran Dasar (Terbaru)', 'Tanggal RAT Terakhir', 'Alamat', 
                 'Kelurahan / Desa', 'Kecamatan', 'Kabupaten', 'Provinsi', 'Bentuk Koperasi', 
                 'Jenis Koperasi', 'Kelompok Koperasi', 'Sektor Usaha', 'DATA PENGURUS', 
                 'Nama Ketua', 'Nama Sekretaris', 'Nama Bendahara', 'DATA KELEMBAGAAN', 
                 'Jumlah Anggota Pria', 'Jumlah Anggota Wanita', 'Total Anggota', 'Total Manajer', 
                 'Total Karyawan', 'DATA LAINNYA', 'Nomor Induk Koperasi (NIK)', 'Status NIK', 
                 'Status Grade ', '\xc2\xa0'])

for i in range (2,12,1): 
    buttons = browser.find_by_xpath('//a[text()="Detil"]') #finding xpath by text-> Detil
    buttons_length = len(buttons) #check length of Detil in paage for looping
    # buttons[0].click()

    #empety list for storing data for looping per page
    d_keterangan = [] #header
    d_datakoperasi = [] #content

    #empety dataframe 0rows 1columns

    for j in range(0, buttons_length):
        move = browser.find_by_xpath('//a[text()="Detil"]')[j].click()

        page = urllib2.urlopen(browser.url) #finding url address
        soup = BeautifulSoup(page, 'html.parser') #parsing the html page

        #find results within table
        table = soup.find('table', id='MainContent_DetailsView1') #finding the table
        results = table.find_all('tr') #finding the tr

        # loop over results
        for result in results:
            # find all columns per result
            data = result.find_all('td')
            # check that columns have data 
            if len(data) == 31: 
                continue

            # write list for variables in rows 2
            datakoperasi = data[1].getText()
            d_datakoperasi.append(datakoperasi)
    
        if len(d_datakoperasi) == 0:
            browser.back() #back to previous windows
            browser.execute_script("window.scrollTo(300, -document.body.scrollHeight);")#mouse scrolling
        else:
            d_datakoperasi = [x.encode('utf-8') for x in d_datakoperasi]  #decode utf-8 
        
            s = pd.Series(d_datakoperasi, index=df.columns)
            df = df.append(s, ignore_index=True)
        
            df.to_csv('koperasi_iterate_perkoperasi.csv')
            
            d_datakoperasi = []

            browser.back() #back to previous windows
            browser.execute_script("window.scrollTo(300, -document.body.scrollHeight);")#mouse scrolling

    a=str(i)
    td = '['+a+']'
    xpath = '//*[@id="MainContent_GridView1"]/tbody/tr[1]/td/table/tbody/tr/td[i]/a'
    x = xpath.replace('[i]',td)
    element = browser.find_by_xpath(x).first 
    element.click()

df.to_csv('koperasi_page1.csv')

i=3
j=[]

while i < 13:
    buttons = browser.find_by_xpath('//a[text()="Detil"]') #finding xpath by text-> Detil
    buttons_length = len(buttons) #check length of Detil in paage for looping
    # buttons[0].click()

    #empety list for storing data for looping per page
    d_keterangan = [] #header
    d_datakoperasi = [] #content

    for j in range(0, buttons_length):
        move = browser.find_by_xpath('//a[text()="Detil"]')[j].click()

        page = urllib2.urlopen(browser.url) #finding url address
        soup = BeautifulSoup(page, 'html.parser') #parsing the html page

        #find results within table
        table = soup.find('table', id='MainContent_DetailsView1') #finding the table
        results = table.find_all('tr') #finding the tr

        # loop over results
        for result in results:
            # find all columns per result
            data = result.find_all('td')
            # check that columns have data 
            if len(data) == 0: 
                continue

            # write list for variables in rows 2
            datakoperasi = data[1].getText()
            d_datakoperasi.append(datakoperasi)

        if len(d_datakoperasi) == 0:
            browser.back() #back to previous windows
            browser.execute_script("window.scrollTo(300, -document.body.scrollHeight);")#mouse scrolling
        else:
            d_datakoperasi = [x.encode('utf-8') for x in d_datakoperasi]  #decode utf-8 
        
            s = pd.Series(d_datakoperasi, index=df.columns)
            df = df.append(s, ignore_index=True)
        
            df.to_csv('koperasi_iterate_perkoperasi.csv')
            
            d_datakoperasi = []

            browser.back() #back to previous windows
            browser.execute_script("window.scrollTo(300, -document.body.scrollHeight);")#mouse scrolling


#     a=str(i)
#     td = '['+a+']'
#     xpath = '//*[@id="MainContent_GridView1"]/tbody/tr[1]/td/table/tbody/tr/td[i]/a'
#     x = xpath.replace('[i]',td)
#     element = browser.find_by_xpath(x).first 
#     element.click()
    
    
    a=str(i)
    td = '['+a+']'
    
    if i < 12:
        df.to_csv('koperasi_iterate_perpage.csv')
        
        xpath = '//*[@id="MainContent_GridView1"]/tbody/tr[1]/td/table/tbody/tr/td[i]/a'
        x = xpath.replace('[i]',td)
    
        element = browser.find_by_xpath(x).first 
        element.click()
   
    else:
        element = browser.find_by_xpath('//*[@id="MainContent_GridView1"]/tbody/tr[1]/td/table/tbody/tr/td[12]/a').first #untuk pilih Kota/Kab
        element.click()
        df.to_csv('koperasi_till_break_12.csv')
        i=2
     
    i+=1
    
print(df)
df.to_csv('koperasi_perkota.csv')
# element = browser.find_by_xpath('//*[@id="MainContent_GridView1"]/tbody/tr[1]/td/table/tbody/tr/td[1]').first #untuk pilih Kota/Kab
# element.click()
# element = browser.find_by_xpath('//*[@id="MainContent_GridView1"]/tbody/tr[1]/td/table/tbody/tr/td[2]').first #untuk pilih Kota/Kab
# element.click()

# buttons = browser.find_by_xpath('//a[text()="Detil"]') #finding xpath by text-> Detil
# buttons_length = len(buttons) #check length of Detil in paage for looping
# # buttons[0].click()

# //*[@id="MainContent_GridView1"]/tbody/tr[1]/td/table/tbody/tr/td[2]/a

# for i in range(0, buttons_length):
#     move = browser.find_by_xpath('//a[text()="Detil"]')[i].click()
#     browser.back() #back to previous windows
#     browser.execute_script("window.scrollTo(200, -document.body.scrollHeight);")#mouse scrolling

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=73.0.3683.86)
  (Driver info: chromedriver=73.0.3683.68 (47787ec04b6e38e22703e856e101e840b65afe72),platform=Mac OS X 10.14.1 x86_64)


In [ ]:
//*[@id="MainContent_GridView1"]/tbody/tr[1]/td/table/tbody/tr/td[8]/a